In [43]:
import pandas as pd
import numpy as np 
from scipy import stats
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings 
from datetime import date, datetime

warnings.filterwarnings('ignore')

### Importing the datasets

In [44]:
data = pd.read_csv("data/online_retail.csv")
data.head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


**COLUMN INSIGHT**
- Index : Purely an internal row counts, no required
- InvoiceNo : 
    - Represents a transaction/order number 
    - Multiple rows share an invoice with each row a line item 
    - Often follows sales order structure
- StockCode : 
    - Likely a product identifier
- Description
    - Product Name 
- Quantity : No. of products bought 
- InvoiceData: Invoice date and time of purchase 
- Unit Price: Selling price for each product 
- CustomerID: Identifies the customer who placed the order
- Country: The country in which the customer resides

In [45]:
data = data.drop(columns="index")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


| Column Name | Current Dtype | Recommended Dtype | Reason               |
| ----------- | ------------- | ----------------- | -------------------- |
| InvoiceNo   | object        | object            | IDs                  |
| StockCode   | object        | object            | categorical ID       |
| Description | object        | object            | text                 |
| Quantity    | int64         | int64             | numeric count        |
| InvoiceDate | object        | **datetime64**    | time series analysis |
| UnitPrice   | float64       | float64           | numeric              |
| CustomerID  | float64       | **object / int**  | ID, not numeric      |
| Country     | object        | object            | categorical          |


Description has 1,454 missing rows 
CustomerID missing 24.93% of rows

In [46]:
# Converting the above columns to the appropriate ones
data.InvoiceDate = pd.to_datetime(data.InvoiceDate)
print(data.InvoiceDate.dtype) 

data.CustomerID = data.CustomerID.astype('object')
print(data.CustomerID.dtype)

datetime64[ns]
object


### Data Statistics

In [47]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853


1. Quantity 
    - minimum value error. Values can only be positive integers greater than 0
    - A huge gap between the minimum quantity and the maximum quantity. So far we can assume they are outliers 
2. Invoice Date 
    - Dataset is for a calender year from 2010-12-07 to 2011-12-09
3. UnitPrice 
    - minimum value error. Values can only be positive integers greater than 0
    - A wide range of prices. We can assume outliers as the 75th percentile is very low compare to max value 

In [48]:
data.describe(include='object').T

,count,unique,top,freq
InvoiceNo,541909,25900,573585,1114
StockCode,541909,4070,85123A,2313
Description,540455,4223,WHITE HANGING HEART T-LIGHT HOLDER,2369
CustomerID,406829.0,4372.0,17841.0,7983.0
Country,541909,38,United Kingdom,495478


1. InvoiceNo 
    - 25,900 Orders with order `573585` being the one with the most number of unique products in the cart (1114, likely a wholesale order)
2. StockCode 
    - The store has 4070 unique products (assuming all were sold at least once in the fiscal year) and product `85123A` is the most popular product in the cart with it being bought 2313 times (!note that we are not speaking of quantities here)
3. Description 
    - A bit of confusion because the assumption is that the stockcode and the description should have the same statistics as it just two ways of presenting the same information however we find that we have 4223 unique products using the description column and `White Hanging Heart T-Light Holder` being the most purchased with 2369 Purchases
4. CustomerID 
    - We have serviced 4372 customers during the fiscal year and customer `17841` has been the most active in terms of number of purchased goods (7983) 
5. Country 
    - We have 38 countries represented, However United Kingdom heavily dominate with 495478 frequency.


## Data Cleaning 

In [49]:
print(f"Zero quantity line items: {round((data[data['Quantity'] == 0].shape[0] / data.shape[0]) * 100, 2)}%")
print(f"Negative quantity line items: {round((data[data['Quantity'] < 0].shape[0] / data.shape[0]) * 100, 2)}%")
print(f"Zero unit price line items: {round((data[data['UnitPrice'] == 0].shape[0] / data.shape[0]) * 100, 2)}%")
print(f"Negative unit price line items: {round((data[data['UnitPrice'] < 0].shape[0] / data.shape[0]) * 100, 2)}%")

Zero quantity line items: 0.0%
Negative quantity line items: 1.96%
Zero unit price line items: 0.46%
Negative unit price line items: 0.0%


In [50]:
data[data['UnitPrice'] == 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom


In [51]:
data[data['StockCode'] == '22139']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
106,536381,22139,RETROSPOT TEA SET CERAMIC 11 PC,23,2010-12-01 09:41:00,4.25,15311.0,United Kingdom
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
6392,536942,22139,amazon,15,2010-12-03 12:08:00,0.00,NaN,United Kingdom
6885,536982,22139,RETROSPOT TEA SET CERAMIC 11 PC,10,2010-12-03 14:27:00,11.02,NaN,United Kingdom
7203,537011,22139,NaN,-5,2010-12-03 15:38:00,0.00,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
538411,581405,22139,RETROSPOT TEA SET CERAMIC 11 PC,1,2011-12-08 13:50:00,4.95,13521.0,United Kingdom
539531,581439,22139,RETROSPOT TEA SET CERAMIC 11 PC,1,2011-12-08 16:30:00,10.79,NaN,United Kingdom
540441,581486,22139,RETROSPOT TEA SET CERAMIC 11 PC,6,2011-12-09 09:38:00,4.95,17001.0,United Kingdom
541387,581498,22139,RETROSPOT TEA SET CERAMIC 11 PC,2,2011-12-09 10:26:00,10.79,NaN,United Kingdom


The product does exist as `RETROSPOT TEA SET CERAMIC 11 PC` and could fill in the missing descriptions however the prices are changing a lot and in a short period of time, for example `$10.79` on the 8th of December and `$4.95` on the 9th of December which leaves us with a big problem of how to impute the Unit Prices. 

Given that it's only 0.46% of the product we will drop them

In [52]:
data2 = data[~(data['UnitPrice'] == 0)]

In [53]:
data2[data2['Quantity'] < 0 ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


Are these returns ? Or any other errors

In [54]:
data2[(data2['CustomerID'] == 15311) & (data2['StockCode'] == '35004C')].sort_values('InvoiceDate')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
9200,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-05 13:55:00,4.65,15311.0,United Kingdom
18196,C537805,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-08 13:18:00,4.65,15311.0,United Kingdom
28630,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-13 15:07:00,4.65,15311.0,United Kingdom
39381,C539640,35004C,SET OF 3 COLOURED FLYING DUCKS,-3,2010-12-20 15:27:00,4.65,15311.0,United Kingdom
44157,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-05 11:41:00,4.65,15311.0,United Kingdom
59542,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-17 13:39:00,4.65,15311.0,United Kingdom
78333,C542866,35004C,SET OF 3 COLOURED FLYING DUCKS,-2,2011-02-01 12:14:00,4.65,15311.0,United Kingdom


Assuming the data starts in the year, having him returning 1 first before purchases shows that these might be more of errors than real returns

In [55]:
discounts = data2[(data2['Description'] == 'Discount')]
df = pd.concat([data2[data2['Quantity'] > 0], discounts])

In [56]:
discounts

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
9038,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527.0,United Kingdom
14498,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498.0,United Kingdom
19392,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340.0,United Kingdom
31134,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422.0,United Kingdom
...,...,...,...,...,...,...,...,...
479868,C577227,D,Discount,-1,2011-11-18 12:06:00,19.82,14527.0,United Kingdom
479869,C577227,D,Discount,-1,2011-11-18 12:06:00,16.76,14527.0,United Kingdom
493613,C578239,D,Discount,-1,2011-11-23 12:29:00,26.33,14912.0,Italy
516221,C579884,D,Discount,-1,2011-11-30 17:34:00,20.53,14527.0,United Kingdom


In [57]:
print(f"Zero quantity line items: {round((df[df['Quantity'] == 0].shape[0] / df.shape[0]) * 100, 2)}%")
print(f"Negative quantity line items: {round((df[df['Quantity'] < 0].shape[0] / df.shape[0]) * 100, 2)}%")
print(f"Zero unit price line items: {round((df[df['UnitPrice'] == 0].shape[0] / df.shape[0]) * 100, 2)}%")
print(f"Negative unit price line items: {round((df[df['UnitPrice'] < 0].shape[0] / df.shape[0]) * 100, 2)}%")

Zero quantity line items: 0.0%
Negative quantity line items: 0.01%
Zero unit price line items: 0.0%
Negative unit price line items: 0.0%


In [58]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,530183.0,10.538218,-720.0,1.0,3.0,10.0,80995.0,155.516307
InvoiceDate,530183,2011-07-04 20:15:24.542166784,2010-12-01 08:26:00,2011-03-28 12:22:00,2011-07-20 12:58:00,2011-10-19 12:39:00,2011-12-09 12:50:00,NaN
UnitPrice,530183.0,3.875841,-11062.06,1.25,2.08,4.13,13541.33,41.943488


In [59]:
df[df['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
9038,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527.0,United Kingdom
14498,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498.0,United Kingdom
19392,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340.0,United Kingdom
31134,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422.0,United Kingdom
...,...,...,...,...,...,...,...,...
479868,C577227,D,Discount,-1,2011-11-18 12:06:00,19.82,14527.0,United Kingdom
479869,C577227,D,Discount,-1,2011-11-18 12:06:00,16.76,14527.0,United Kingdom
493613,C578239,D,Discount,-1,2011-11-23 12:29:00,26.33,14912.0,Italy
516221,C579884,D,Discount,-1,2011-11-30 17:34:00,20.53,14527.0,United Kingdom


In [60]:
df[df['InvoiceNo'] == 'C578239']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
493613,C578239,D,Discount,-1,2011-11-23 12:29:00,26.33,14912.0,Italy


Because the discounts are not tied to any invoice now we have to take them away too

In [61]:
df2 = df[df['Quantity'] > 0]
df2.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,530106.0,10.542001,1.0,1.0,3.0,10.0,80995.0,155.523831
InvoiceDate,530106,2011-07-04 20:16:17.864539392,2010-12-01 08:26:00,2011-03-28 12:22:00,2011-07-20 12:58:00,2011-10-19 12:39:00,2011-12-09 12:50:00,NaN
UnitPrice,530106.0,3.865875,-11062.06,1.25,2.08,4.13,13541.33,41.85612


We still have negative UnitPrice that needs to be fixed

In [62]:
df2[df2['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [63]:
df_final = df2[df2['UnitPrice'] > 0]

In [64]:
df_final.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,530104.0,10.542037,1.0,1.0,3.0,10.0,80995.0,155.524124
InvoiceDate,530104,2011-07-04 20:16:05.225087744,2010-12-01 08:26:00,2011-03-28 12:22:00,2011-07-20 12:58:00,2011-10-19 12:39:00,2011-12-09 12:50:00,NaN
UnitPrice,530104.0,3.907625,0.001,1.25,2.08,4.13,13541.33,35.915681


In [65]:
df_final[df_final['Quantity'] > 1000]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4850,536809,84950,ASSORTED COLOUR T-LIGHT HOLDER,1824,2010-12-02 16:48:00,0.55,15299.0,United Kingdom
4945,536830,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,2880,2010-12-02 17:38:00,0.18,16754.0,United Kingdom
4946,536830,21915,RED HARMONICA IN BOX,1400,2010-12-02 17:38:00,1.06,16754.0,United Kingdom
6365,536890,17084R,ASSORTED INCENSE PACK,1440,2010-12-03 11:48:00,0.16,14156.0,EIRE
16435,537659,22189,CREAM HEART CARD HOLDER,1008,2010-12-07 16:43:00,2.31,18102.0,United Kingdom
...,...,...,...,...,...,...,...,...
533812,581115,22413,METAL SIGN TAKE IT OR LEAVE IT,1404,2011-12-07 12:20:00,2.75,15195.0,United Kingdom
534952,581175,23084,RABBIT NIGHT LIGHT,1440,2011-12-07 15:16:00,1.79,14646.0,Netherlands
540070,581458,22197,POPCORN HOLDER,1500,2011-12-08 18:45:00,0.72,17949.0,United Kingdom
540071,581459,22197,POPCORN HOLDER,1200,2011-12-08 18:46:00,0.72,17949.0,United Kingdom


Given the prices I think these quantities are justifiable so we will leave them.

In [66]:
df_final[df_final['UnitPrice'] > 1000]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
15017,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom
41448,539856,M,Manual,1,2010-12-22 14:41:00,1298.40,NaN,United Kingdom
117054,546328,M,Manual,1,2011-03-11 10:19:00,1687.17,14911.0,EIRE
117055,546329,M,Manual,1,2011-03-11 10:22:00,1687.17,14911.0,EIRE
119632,546558,M,Manual,1,2011-03-15 09:50:00,2583.76,NaN,Hong Kong
144826,548813,M,Manual,1,2011-04-04 13:03:00,2382.92,12744.0,Singapore
144828,548813,M,Manual,1,2011-04-04 13:03:00,1252.95,12744.0,Singapore
144829,548820,M,Manual,1,2011-04-04 13:04:00,2053.07,12744.0,Singapore
145831,548913,M,Manual,1,2011-04-05 09:45:00,1136.30,12669.0,France
150615,549468,M,Manual,1,2011-04-08 14:27:00,1867.86,17940.0,United Kingdom


In [67]:
financial_codes = ['M', 'POST', 'DOT', 'DOTCOM POSTAGE', 'DOT POSTAGE', 
                   'AMAZONFEE', 'B']

data_clean = df_final[~df_final['StockCode'].isin(financial_codes)]

data_clean[data_clean['UnitPrice'] > 500]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.5,15098.0,United Kingdom
222682,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.5,15098.0,United Kingdom


In [68]:
data_clean.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,527948.0,10.564095,1.0,1.0,3.0,11.0,80995.0,155.806478
InvoiceDate,527948,2011-07-04 21:28:10.461030656,2010-12-01 08:26:00,2011-03-28 12:23:00,2011-07-20 13:26:00,2011-10-19 13:38:00,2011-12-09 12:50:00,NaN
UnitPrice,527948.0,3.279257,0.001,1.25,2.08,4.13,649.5,4.449435


In [69]:
data_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [70]:
data_clean['line_total'] = data_clean['Quantity'] * data_clean['UnitPrice']

In [71]:
data_clean['cart_total_amount'] = data_clean.groupby(['CustomerID', 'InvoiceNo'])['UnitPrice'].transform('sum')
data_clean['cart_total_products'] = data_clean.groupby(['CustomerID', 'InvoiceNo'])['StockCode'].transform('nunique')
data_clean['cart_total_units'] = data_clean.groupby(['CustomerID', 'InvoiceNo'])['Quantity'].transform('sum')
data_clean['first_date_of_purchase'] = data_clean.groupby(['CustomerID'])['InvoiceDate'].transform('min')
data_clean['last_date_of_purchase'] = data_clean.groupby(['CustomerID'])['InvoiceDate'].transform('max')
data_clean['recency'] = max(data_clean['InvoiceDate']) - data_clean['last_date_of_purchase']
data_clean['customer_tenure'] = data_clean['last_date_of_purchase'] - data_clean['first_date_of_purchase']
data_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00


In [72]:
data_clean['StockCode'] = data_clean['StockCode'].str.upper()

In [73]:
def create_clean_product_mapping(df):
    """
    Create clean product mapping, removing administrative and problematic entries
    """
    filtered_data = df[
        df['Description'].notna() &
        (df['Description'].str.len() > 5) &
        (~df['StockCode'].astype(str).str.match(r'^[A-Za-z]$')) &
        (~df['StockCode'].isin(['POST', 'D', 'C2', 'DOT', 'BANK CHARGES', 'M', 'B', 'S', 'PADS'])) &
        (~df['Description'].str.contains(
            r'damage|check|incorrect|thrown away|missing|found|lost|crushed|wet|rusty|mouldy|adjustment|test',
            case=False, na=False
        )) &
        (~df['Description'].str.contains(r'\?\?+', na=False)) &
        (~df['Description'].str.lower().isin(['dotcom', 'amazon', 'manual', 'discount']))
    ]
    
    # Group by StockCode and get most common description
    product_mapping = (
        filtered_data
        .groupby('StockCode')['Description']
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
        .dropna()
        .to_dict()
    )
    
    return product_mapping

# Usage
actual_product_mapping = create_clean_product_mapping(data_clean)

In [74]:
data_clean['NewDescription'] = data_clean['StockCode'].map(actual_product_mapping)

In [75]:
data_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure,NewDescription
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE METAL LANTERN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,PACK OF 20 SPACEBOY NAPKINS
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDREN'S APRON DOLLY GIRL
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY DOLLY GIRL
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY CIRCUS PARADE


## ANALYSIS

In [76]:
data_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure,NewDescription
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE METAL LANTERN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,PACK OF 20 SPACEBOY NAPKINS
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDREN'S APRON DOLLY GIRL
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY DOLLY GIRL
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY CIRCUS PARADE


In [78]:
products = data_clean.groupby(['StockCode', 'NewDescription']).agg(
    TotalSales=('line_total', 'sum'), 
    TotalVolume=('Quantity', 'sum'), 
    UnitPrice=('UnitPrice', 'mean'), 
    CustomerInteraction=('CustomerID', 'nunique'), 
    CountryChannels=('Country', 'nunique'), 
    FirstPurchasedDate=('InvoiceDate', 'first'), 
    LastPurchasedDate=('InvoiceDate', 'last')
).reset_index()

In [79]:
products

,StockCode,NewDescription,TotalSales,TotalVolume,UnitPrice,CustomerInteraction,CountryChannels,FirstPurchasedDate,LastPurchasedDate
0,10002,INFLATABLE POLITICAL GLOBE,759.89,860,1.086620,40,7,2010-12-01 08:45:00,2011-04-18 12:56:00
1,10080,GROOVY CACTUS INFLATABLE,119.09,303,0.410909,19,1,2011-02-27 13:47:00,2011-11-21 17:04:00
2,10120,DOGGY RUBBER,40.53,193,0.210000,25,2,2010-12-03 11:19:00,2011-12-04 13:15:00
3,10123C,HEARTS WRAPPING TAPE,3.25,5,0.650000,3,1,2010-12-03 11:19:00,2011-03-31 13:14:00
4,10124A,SPOTS ON RED BOOKCOVER TAPE,6.72,16,0.420000,5,1,2010-12-06 13:13:00,2011-11-06 13:00:00
...,...,...,...,...,...,...,...,...,...
3793,GIFT_0001_10,Dotcomgiftshop Gift Voucher £10.00,74.97,9,8.330000,0,1,2011-03-07 17:14:00,2011-12-08 09:20:00
3794,GIFT_0001_20,Dotcomgiftshop Gift Voucher £20.00,167.05,10,16.708889,0,1,2011-01-05 14:44:00,2011-10-31 14:41:00
3795,GIFT_0001_30,Dotcomgiftshop Gift Voucher £30.00,175.53,7,25.075714,0,1,2011-01-05 14:44:00,2011-09-02 09:26:00
3796,GIFT_0001_40,Dotcomgiftshop Gift Voucher £40.00,100.70,3,33.566667,0,1,2010-12-20 10:14:00,2011-07-27 15:12:00


In [80]:
customers = data_clean.groupby('CustomerID').agg(
    TotalSales=('line_total', 'sum'), 
    TotalVolume=('Quantity', 'sum'), 
    TotalOrders=('InvoiceNo', 'nunique'), 
    ProductsInteractedWith=('StockCode', 'nunique'), 
    FirstPurchasedDate=('InvoiceDate', 'first'), 
    LastPurchasedDate=('InvoiceDate', 'last')  
).reset_index()
customers['AOV'] = customers['TotalSales'] / customers['TotalOrders']

In [81]:
customers

,CustomerID,TotalSales,TotalVolume,TotalOrders,ProductsInteractedWith,FirstPurchasedDate,LastPurchasedDate,AOV
0,12346.0,77183.60,74215,1,1,2011-01-18 10:01:00,2011-01-18 10:01:00,77183.600000
1,12347.0,4310.00,2458,7,103,2010-12-07 14:57:00,2011-12-07 15:52:00,615.714286
2,12348.0,1437.24,2332,4,21,2010-12-16 19:09:00,2011-09-25 13:13:00,359.310000
3,12349.0,1457.55,630,1,72,2011-11-21 09:51:00,2011-11-21 09:51:00,1457.550000
4,12350.0,294.40,196,1,16,2011-02-02 16:01:00,2011-02-02 16:01:00,294.400000
...,...,...,...,...,...,...,...,...
4330,18280.0,180.60,45,1,10,2011-03-07 09:52:00,2011-03-07 09:52:00,180.600000
4331,18281.0,80.82,54,1,7,2011-06-12 10:53:00,2011-06-12 10:53:00,80.820000
4332,18282.0,178.05,103,2,12,2011-08-05 13:35:00,2011-12-02 11:43:00,89.025000
4333,18283.0,2088.93,1395,16,262,2011-01-06 14:14:00,2011-12-06 12:02:00,130.558125


In [82]:
data_clean['ActiveMonth'] = data_clean['InvoiceDate'].dt.strftime('%b-%Y')
data_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure,NewDescription,ActiveMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE HANGING HEART T-LIGHT HOLDER,Dec-2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE METAL LANTERN,Dec-2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,CREAM CUPID HEARTS COAT HANGER,Dec-2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,KNITTED UNION FLAG HOT WATER BOTTLE,Dec-2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,RED WOOLLY HOTTIE WHITE HEART.,Dec-2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,PACK OF 20 SPACEBOY NAPKINS,Dec-2011
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDREN'S APRON DOLLY GIRL,Dec-2011
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY DOLLY GIRL,Dec-2011
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY CIRCUS PARADE,Dec-2011


In [83]:
customers['ActiveMonths'] = data_clean.groupby('CustomerID')['ActiveMonth'].transform('nunique')
customers

,CustomerID,TotalSales,TotalVolume,TotalOrders,ProductsInteractedWith,FirstPurchasedDate,LastPurchasedDate,AOV,ActiveMonths
0,12346.0,77183.60,74215,1,1,2011-01-18 10:01:00,2011-01-18 10:01:00,77183.600000,1.0
1,12347.0,4310.00,2458,7,103,2010-12-07 14:57:00,2011-12-07 15:52:00,615.714286,1.0
2,12348.0,1437.24,2332,4,21,2010-12-16 19:09:00,2011-09-25 13:13:00,359.310000,1.0
3,12349.0,1457.55,630,1,72,2011-11-21 09:51:00,2011-11-21 09:51:00,1457.550000,1.0
4,12350.0,294.40,196,1,16,2011-02-02 16:01:00,2011-02-02 16:01:00,294.400000,1.0
...,...,...,...,...,...,...,...,...,...
4330,18280.0,180.60,45,1,10,2011-03-07 09:52:00,2011-03-07 09:52:00,180.600000,8.0
4331,18281.0,80.82,54,1,7,2011-06-12 10:53:00,2011-06-12 10:53:00,80.820000,8.0
4332,18282.0,178.05,103,2,12,2011-08-05 13:35:00,2011-12-02 11:43:00,89.025000,8.0
4333,18283.0,2088.93,1395,16,262,2011-01-06 14:14:00,2011-12-06 12:02:00,130.558125,8.0


In [84]:
country_map = (
    data_clean.groupby('CustomerID')['Country']
    .agg(lambda x: x.mode().iloc[0])
)

customers['Country'] = customers['CustomerID'].map(country_map)


In [85]:
customers

,CustomerID,TotalSales,TotalVolume,TotalOrders,ProductsInteractedWith,FirstPurchasedDate,LastPurchasedDate,AOV,ActiveMonths,Country
0,12346.0,77183.60,74215,1,1,2011-01-18 10:01:00,2011-01-18 10:01:00,77183.600000,1.0,United Kingdom
1,12347.0,4310.00,2458,7,103,2010-12-07 14:57:00,2011-12-07 15:52:00,615.714286,1.0,Iceland
2,12348.0,1437.24,2332,4,21,2010-12-16 19:09:00,2011-09-25 13:13:00,359.310000,1.0,Finland
3,12349.0,1457.55,630,1,72,2011-11-21 09:51:00,2011-11-21 09:51:00,1457.550000,1.0,Italy
4,12350.0,294.40,196,1,16,2011-02-02 16:01:00,2011-02-02 16:01:00,294.400000,1.0,Norway
...,...,...,...,...,...,...,...,...,...,...
4330,18280.0,180.60,45,1,10,2011-03-07 09:52:00,2011-03-07 09:52:00,180.600000,8.0,United Kingdom
4331,18281.0,80.82,54,1,7,2011-06-12 10:53:00,2011-06-12 10:53:00,80.820000,8.0,United Kingdom
4332,18282.0,178.05,103,2,12,2011-08-05 13:35:00,2011-12-02 11:43:00,89.025000,8.0,United Kingdom
4333,18283.0,2088.93,1395,16,262,2011-01-06 14:14:00,2011-12-06 12:02:00,130.558125,8.0,United Kingdom


In [86]:
data_clean[data_clean['CustomerID'].isna()].head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure,NewDescription,ActiveMonth
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom,2.51,NaN,NaN,NaN,NaT,NaT,NaT,NaT,DECORATIVE ROSE BATHROOM BOTTLE,Dec-2010
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,5.02,NaN,NaN,NaN,NaT,NaT,NaT,NaT,DECORATIVE CATS BATHROOM BOTTLE,Dec-2010


In [87]:
data_clean[~(data_clean['CustomerID'].isna())]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,line_total,cart_total_amount,cart_total_products,cart_total_units,first_date_of_purchase,last_date_of_purchase,recency,customer_tenure,NewDescription,ActiveMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE HANGING HEART T-LIGHT HOLDER,Dec-2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,WHITE METAL LANTERN,Dec-2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,CREAM CUPID HEARTS COAT HANGER,Dec-2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,KNITTED UNION FLAG HOT WATER BOTTLE,Dec-2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,27.37,7.0,40.0,2010-12-01 08:26:00,2010-12-02 15:27:00,371 days 21:23:00,1 days 07:01:00,RED WOOLLY HOTTIE WHITE HEART.,Dec-2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,PACK OF 20 SPACEBOY NAPKINS,Dec-2011
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDREN'S APRON DOLLY GIRL,Dec-2011
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY DOLLY GIRL,Dec-2011
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,44.50,15.0,105.0,2011-08-18 15:44:00,2011-12-09 12:50:00,0 days 00:00:00,112 days 21:06:00,CHILDRENS CUTLERY CIRCUS PARADE,Dec-2011


In [88]:
customers.to_csv('data/clean_data/customers.csv', index=False)
data_clean.to_csv('data/clean_data/online_retail_data.csv', index=False)
products.to_csv('data/clean_data/products.csv', index=False)